In [3]:
import json
import sqlite3
import urllib.request
import time
from datetime import datetime
import pprint
import string

In [4]:
#open database
conn = sqlite3.connect('log.db')
c = conn.cursor()

c.execute("CREATE TABLE IF NOT EXISTS log_802 (id INTEGER PRIMARY KEY AUTOINCREMENT, timestamp DATETIME NOT NULL, vehicle_id INTEGER NOT NULL, route_id INTEGER NOT NULL, lat REAL NOT NULL, lon REAL NOT NULL, direction REAL NOT NULL)")

# get vehicles data from API
req = urllib.request.Request('http://api.metro.net/agencies/lametro-rail/routes/801/vehicles/')
req.add_header('Accept', 'application/json')
response = urllib.request.urlopen(req)
data = response.read()
pp = str(data)
print(pp)


b'{\n    "items": [\n        {\n            "latitude": 34.038428,\n            "id": "105",\n            "run_id": "801_0_var0",\n            "predictable": true,\n            "seconds_since_report": 66,\n            "longitude": -118.267781,\n            "route_id": "801",\n            "heading": 0.0\n        },\n        {\n            "latitude": 33.8916449,\n            "id": "122",\n            "run_id": "801_1_var2",\n            "predictable": true,\n            "seconds_since_report": 8,\n            "longitude": -118.223814,\n            "route_id": "801",\n            "heading": 180.0\n        },\n        {\n            "latitude": 34.01928,\n            "id": "104",\n            "run_id": "801_0_var0",\n            "predictable": true,\n            "seconds_since_report": 66,\n            "longitude": -118.24305,\n            "route_id": "801",\n            "heading": 0.0\n        },\n        {\n            "latitude": 33.931796,\n            "id": "106",\n            "run_i

In [5]:
# derive timestamp from datetime string in returned header
time_string = response.info()['Date']
date_obj = datetime.strptime(time_string, '%a, %d %b %Y %H:%M:%S %Z')
ts = date_obj.timestamp()

# load data into dict object and access array of items
parsed = json.loads(data)
items = parsed["items"]
pprint.pprint(items)
# these fields must be present in each item for data to be valid
fields = ['id', 'seconds_since_report', 'route_id', 'latitude', 'longitude', 'heading']
# list of valid items
valid = []
# check all fields are present
for vehicle in items:
    for field in fields:
        if not field in vehicle.keys():
            all_fields_present = False
            break
        else:
            all_fields_present = True
    if all_fields_present == True:
        # push vehicle into valid array
        valid.append(vehicle)

for vehicle in valid:
    # calculate timestamp of each position entry and add key to dict
    vehicle['timestamp'] = ts - int(vehicle['seconds_since_report'])
    # insert each vehicle position entry into database
    c.execute("INSERT INTO log_802 (timestamp, vehicle_id, route_id, lat, lon, direction) VALUES (?, ?, ?, ?, ?, ?)", [float(vehicle['timestamp']), int(vehicle['id']),int(vehicle['route_id']), float(vehicle['latitude']), float(vehicle['longitude']), float(vehicle['heading'])])

conn.commit()
conn.close()
print("DONE")

[{'heading': 0.0,
  'id': '105',
  'latitude': 34.038428,
  'longitude': -118.267781,
  'predictable': True,
  'route_id': '801',
  'run_id': '801_0_var0',
  'seconds_since_report': 66},
 {'heading': 180.0,
  'id': '122',
  'latitude': 33.8916449,
  'longitude': -118.223814,
  'predictable': True,
  'route_id': '801',
  'run_id': '801_1_var2',
  'seconds_since_report': 8},
 {'heading': 0.0,
  'id': '104',
  'latitude': 34.01928,
  'longitude': -118.24305,
  'predictable': True,
  'route_id': '801',
  'run_id': '801_0_var0',
  'seconds_since_report': 66},
 {'heading': 0.0,
  'id': '106',
  'latitude': 33.931796,
  'longitude': -118.239443,
  'predictable': True,
  'route_id': '801',
  'run_id': '801_0_var0',
  'seconds_since_report': 36},
 {'heading': 180.0,
  'id': '124',
  'latitude': 33.992585,
  'longitude': -118.243433,
  'predictable': True,
  'route_id': '801',
  'run_id': '801_1_var2',
  'seconds_since_report': 55},
 {'heading': 180.0,
  'id': '107',
  'latitude': 33.959611,
  '

OperationalError: database is locked